# Day 9: Encoding Error

With your neighbor happily enjoying their video game, you turn your attention to an open data port on the little screen in the seat in front of you.

Though the port is non-standard, you manage to connect it to your computer through the clever use of several paperclips. Upon connection, the port outputs a series of numbers (your puzzle input).

The data appears to be encrypted with the eXchange-Masking Addition System (XMAS) which, conveniently for you, is an old cypher with an important weakness.

XMAS starts by transmitting a preamble of 25 numbers. After that, each number you receive should be the sum of any two of the 25 immediately previous numbers. The two numbers will have different values, and there might be more than one such pair.

For example, suppose your preamble consists of the numbers 1 through 25 in a random order. To be valid, the next number must be the sum of two of those numbers:

    26 would be a valid next number, as it could be 1 plus 25 (or many other pairs, like 2 and 24).
    49 would be a valid next number, as it is the sum of 24 and 25.
    100 would not be valid; no two of the previous 25 numbers sum to 100.
    50 would also not be valid; although 25 appears in the previous 25 numbers, the two numbers in the pair must be different.

Suppose the 26th number is 45, and the first number (no longer an option, as it is more than 25 numbers ago) was 20. Now, for the next number to be valid, there needs to be some pair of numbers among 1-19, 21-25, or 45 that add up to it:

    26 would still be a valid next number, as 1 and 25 are still within the previous 25 numbers.
    65 would not be valid, as no two of the available numbers sum to it.
    64 and 66 would both be valid, as they are the result of 19+45 and 21+45 respectively.

Here is a larger example which only considers the previous 5 numbers (and has a preamble of length 5):

```text
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```

In this example, after the 5-number preamble, almost every number is the sum of two of the previous 5 numbers; the only number that does not follow this rule is 127.

The first step of attacking the weakness in the XMAS data is to find the first number in the list (after the preamble) which is not the sum of two of the 25 numbers before it. What is the first number that does not have this property?

In [1]:
# Python imports
from itertools import combinations
from pathlib import Path
from typing import List

For convenience, we write a function that loads the XMAS data as a list of integers (one per line of the input file).

In [2]:
def load_xmas_data(fpath: str) -> List[int]:
    """Load XMAS data as a list of ints.
    
    :param fpath:  path to the input XMAS data file
    """
    with Path(fpath).open("r") as ifh:
        return [int(_.strip()) for _ in ifh.readlines()]

To find the invalid number, we first define a window of elements in the data of the same length as the preamble size, starting at the first element. This is, to begin with, the preamble. This set of numbers will never be reused, so we can split the list between the preamble and the rest of the data.

We find the sum of all combinations of values in this window, using `itertools.combinations()` as a list.

If the first value in the (rest of the) data is in the list, the number is valid and we can move on. To do this, we *pop* the first value off our window, and extend the list with the first value of the remaining data (the number we just checked) *pop*ped off the remaining data list.

However, if the first value in the (rest of the) data is not in the list, it is the invalid number we're looking for, and the function returns it.

In [3]:
def find_xmas_invalid_number(data: List[int], preamble_size: int) -> int:
    """Return first invalid number in the passed XMAS data
    
    :param data:  input XMAS data
    :param preamble_size:  the number of values in the preamble (and subsequent window)
    """
    window = data[:preamble_size]
    data = data[preamble_size:]
    
    while len(data):
        if data[0] not in (sum(_) for _ in combinations(window, 2)):
            return data[0]
        else:
            window.pop(0)
            window.append(data.pop(0))

Solving the test puzzle.

In [4]:
data = load_xmas_data("day09_test.txt")
find_xmas_invalid_number(data, 5)

127

Solving the real puzzle.

In [5]:
data = load_xmas_data("day09_data.txt")
find_xmas_invalid_number(data, 25)

552655238

## Puzzle 2

The final step in breaking the XMAS encryption relies on the invalid number you just found: you must find a contiguous set of at least two numbers in your list which sum to the invalid number from step 1.

Again consider the above example:

```text
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
```

In this list, adding up all of the numbers from 15 through 40 produces the invalid number from step 1, 127. (Of course, the contiguous set of numbers in your actual list might be much longer.)

To find the encryption weakness, add together the smallest and largest number in this contiguous range; in this example, these are 15 and 47, producing 62.

What is the encryption weakness in your XMAS-encrypted list of numbers?

To find the XMAS weakness in the data we write a function that first finds the corresponding invalid number.

Next, we define a pointer `ptr` to an index in the data (starting from 0, the first position). We then check every window size in the data (starting from that pointer) from 2 upwards in turn, for whether it sums to a value greater than the invalid number. If it does, we move to the next pointer position and repeat the process. This takes advantage of the observation that, once the window elements first sum to a certain value - e.g. `winsum`, because the next value is a positive integer, any larger window must sum to an even larger value.

If the window does not sum to a value greater than the invalid number, we check whether it sums to the invalid number itself. If it does, then we 

In [6]:
def find_xmas_weakness(data: List[int], preamble_size: int) -> int:
    """Return encryption weakness of passed XMAS data
    
    :param data:  input XMAS data
    :preamble_size:  the number of values in the preamble (and subsequence window)
    """
    invalid = find_xmas_invalid_number(data, preamble_size)
    
    ptr = 0
    while ptr < len(data):
        for winsize in range(2, len(data) - ptr):
            window = data[ptr:ptr+winsize]
            winsum = sum(window)
            if winsum > invalid:
                break
            if winsum == invalid:
                return min(window) + max(window)
        ptr += 1

In [7]:
data = load_xmas_data("day09_test.txt")
find_xmas_weakness(data, 5)

62

In [8]:
data = load_xmas_data("day09_data.txt")
find_xmas_weakness(data, 25)

70672245